# **Sesión 1:** Calibración de Cámara 📷⚙️

## **Instalaciones**

In [38]:
!pip install numpy==1.26 opencv-python==4.8.0.76 imageio

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


## **Librerías**

In [39]:
import cv2
import os
import cv2
import numpy as np
import glob
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
from typing import List
import imageio

## **Apartado A: Calibración de cámara** (derecha e izquierda)

### **A.1:** Captura de imagenes.

In [40]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

In [41]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = [f for f in glob.glob("../data/Imagen*.jpg")]
print(f"Found {len(imgs_path)} images")
imgs = load_images(imgs_path)
print(f"Loaded {len(imgs)} images")

Found 38 images


Loaded 38 images


### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [42]:
# TODO Find corners with cv2.findChessboardCorners()
corners = [cv2.findChessboardCorners(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (7, 7), None) for img in imgs]
print(f'Found corners in {len(corners)} images')

Found corners in 38 images


In [43]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (7, 7), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]
print(f'Refined corners in {len(corners_refined)} images')

Refined corners in 38 images


### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [44]:
imgs_copy = copy.deepcopy(imgs)

In [45]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
for i, img in enumerate(imgs_copy):
    if corners[i][0]:
        cv2.drawChessboardCorners(img, (7, 7), corners_refined[i], True)
print('Corners drawn')

Corners drawn


In [46]:
# Crear Carpeta
if not os.path.exists("../Result"):
    os.makedirs("../Result")

In [47]:
# TODO Show images and save when needed

def show_image(img, title="Image", show=False):
    if show:
        cv2.imshow(title, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
def write_image(filename, img):
    cv2.imwrite(filename, img)

show_images = False
for i, img in enumerate(imgs_copy):
    show_image(img, f"Image {i}", show_images)
    write_image(f"../Result/Image_{i}.jpg", img)

### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [48]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    rows, columns = chessboard_shape
    
    objp = np.zeros((rows * columns, 3), np.float32)
    
    objp[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
    
    objp[:, 0] *= dx
    objp[:, 1] *= dy 
    
    return objp

print('Chessboard points calculated')


Chessboard points calculated


In [49]:
# TODO You need the points for every image, not just one
chessboard_points = get_chessboard_points((7, 7), 30, 30)

objpoints = [chessboard_points for _ in range(len(imgs))]

### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [50]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [51]:
image_size = imgs[0].shape[1::-1]
valid_objpoints = []
valid_corners_filtered = []

for objp, corners in zip(objpoints, valid_corners):
    if corners is not None:
        valid_objpoints.append(objp)
        valid_corners_filtered.append(corners)
# TODO
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    valid_objpoints,
    valid_corners_filtered,
    image_size,
    None,
    None
)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))


In [52]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[2.89295437e+04 0.00000000e+00 6.38394349e+02]
 [0.00000000e+00 1.91301639e+04 3.58683444e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[50.50349858  2.28122837  0.18564972  0.07040023  0.0570115 ]]
Root mean squared reprojection error:
 1.3782918368913035


## **Apartado B: Corrección de distorsión** (ojo de pez)

En este apartado se trabajará en la corrección de la distorsión debido a lentes de ojo de pez. Primero se calibrará una cámara con este tipo de lente, utilizando las imágenes de la carpeta ``fisheye``. Posteriormente se utilizarán estos parámetros de calibración para corregir la distorsión de una de las imágenes de calibración.

Los pasos que deberá seguir para calibrar una cámara con distorsión de ojo de pez son:

1. Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``.
2. Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.
3. Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero.
4. Defina los argumentos para la función de calibración.
5. Calibre con ``cv2.fisheye.calibrate()``

### **Tarea B.1:** Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``

In [ ]:


# TODO Build a list containing the paths of all images from the fisheye camera and load images
fisheye_imgs_path = glob.glob('../data/fisheye/*.jpg')
fisheye_imgs = load_images(fisheye_imgs_path)
print(f'Loaded {len(fisheye_imgs)} fisheye images')
print(fisheye_imgs_path)
# print(f'{fisheye_imgs}')

Loaded 10 fisheye images
['../data/fisheye/VMRImage8.jpg', '../data/fisheye/VMRImage1.jpg', '../data/fisheye/VMRImage9.jpg', '../data/fisheye/VMRImage5.jpg', '../data/fisheye/VMRImage3.jpg', '../data/fisheye/VMRImage0.jpg', '../data/fisheye/VMRImage4.jpg', '../data/fisheye/VMRImage2.jpg', '../data/fisheye/VMRImage6.jpg', '../data/fisheye/VMRImage7.jpg']


/tmp/ipykernel_182370/283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


### **Tarea B.2:** Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.

In [ ]:
imgs_corners = []
# Parameters for cv2.cornerSubPix()
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
chessboard_size = (7, 6)
#TODO Complete the required parts of the loop
for img in fisheye_imgs:
    
    # TODO parse arguments to cv2.findChessboardCorners()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray_img, chessboard_size, None)
    
    if ret:
        refined_corners = cv2.cornerSubPix(gray_img, corners, (3, 3), (-1, -1), subpix_criteria)
        
        imgs_corners.append(refined_corners)
    else:
        imgs_corners.append(None)
        
print(f'Número de imágenes procesadas: {len(imgs_corners)}')
print(f'Número de esquinas detectadas correctamente: {len([c for c in imgs_corners if c is not None])}')
# print(imgs_corners)

Número de imágenes procesadas: 10
Número de esquinas detectadas correctamente: 10


### **Tarea B.3:** Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero

In [ ]:
# TODO Define the chessboard dimensions and the lenght of the squares (in [mm])
chessboard_dims = (7, 6)
length = 30
# TODO You need the points for every image, not just one (consider a list comprehension)
fisheye_chessboard_points = [get_chessboard_points(chessboard_dims, length, length) for _ in range(len(fisheye_imgs))]
# print(f'{fisheye_chessboard_points}')

### **Tarea B.4:** Defina los argumentos para la calibración

In [ ]:
# Parameters for cv2.fisheye.calibrate()
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
intrinsics = np.zeros((3, 3))
distortion = np.zeros((4, 1))
rotations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]
traslations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]


### **Tarea B.5:** Calibración

In [ ]:
fisheye_chessboard_points = [np.array(points, dtype=np.float32).reshape(-1, 1, 3) for points in fisheye_chessboard_points]

imgs_corners = [np.array(corners, dtype=np.float32).reshape(-1, 1, 2) for corners in imgs_corners if corners is not None]

print(f"Tipo de datos de fisheye_chessboard_points después de la conversión: {type(fisheye_chessboard_points)}")
print(f"Forma de fisheye_chessboard_points después de la conversión: {np.array(fisheye_chessboard_points).shape}")
print(f"Tipo de datos de imgs_corners después de la conversión: {type(imgs_corners)}")
print(f"Forma de imgs_corners después de la conversión: {np.array(imgs_corners).shape}")

try:
    rms, intrinsics, distortion, rotations, traslations = cv2.fisheye.calibrate(
        fisheye_chessboard_points,
        imgs_corners,
        gray_img.shape[::-1],
        intrinsics,
        distortion,
        rotations,
        traslations,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )

    # Mostrar los resultados de la calibración
    print(f"Error RMS: {rms}")
    print(f"Matriz intrínseca:\n{intrinsics}")
    print(f"Coeficientes de distorsión:\n{distortion}")
except cv2.error as e:
    print(f"Error en la calibración: {e}")


Tipo de datos de fisheye_chessboard_points después de la conversión: <class 'list'>
Forma de fisheye_chessboard_points después de la conversión: (10, 42, 1, 3)
Tipo de datos de imgs_corners después de la conversión: <class 'list'>
Forma de imgs_corners después de la conversión: (10, 42, 1, 2)
Error en la calibración: OpenCV(4.8.0) /io/opencv/modules/calib3d/src/fisheye.cpp:757: error: (-215:Assertion failed) rvecs.empty() || (rvecs.channels() == 3) in function 'calibrate'



In [ ]:
# Show intrinsic matrix and distortion coefficients values
print(intrinsics)
print(distortion)

[[188.86914026   0.         503.92900347]
 [  0.         185.32638678 373.58044468]
 [  0.           0.           1.        ]]
[[ 0.07734388]
 [ 0.01482361]
 [ 0.02351869]
 [-0.02757565]]


### **Pregunta B.1:** Corrija la distorsión de las 2 primeras imágenes de la carpeta ``fisheye``

In [ ]:
# TODO Search in the documentation to define 'dim'
dim = fisheye_imgs[0].shape[1::-1]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(intrinsics, distortion, np.eye(3), intrinsics, dim, cv2.CV_16SC2)

In [ ]:
# TODO Homework: correct distortion using cv2.remap()

undistorted_imgs = []

num_imgs_to_correct = 2

for i, img in enumerate(fisheye_imgs[:num_imgs_to_correct]):
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    
    undistorted_imgs.append(undistorted_img)
    
    cv2.imwrite(f'undistorted_fisheye_image_{i+1}.jpg', undistorted_img)
    
    # (Descomentar para mostrar)
    # cv2.imshow(f'Undistorted Image {i+1}', undistorted_img)
    # cv2.waitKey(2000)  # Mostrar la imagen durante 500ms (ajustable)

cv2.destroyAllWindows()